In [1]:

%load_ext autoreload
%autoreload 2
import json, os, cv2
from pathlib import Path
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import xarray as xr
import seaborn as sns
import h5py as hf
from tqdm import tqdm
from tqdm import tqdm
import itertools 
from scipy.interpolate import interp1d
from scipy import signal
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.cluster import KMeans
import matplotlib.colors as mcolors
import os, fnmatch
from scipy.spatial.distance import cdist
import matplotlib.gridspec as gridspec
from random import sample
from scipy.ndimage import gaussian_filter1d
from matplotlib.collections import LineCollection


import sys
sys.path.insert(0, 'C:/Users/nlab/Documents/GitHub/obstacle_avoidance')

from utils.base_functions import *
from src.utils.auxiliary import flatten_series
from src.utils.path import find
from src.base import BaseInput
from plots.plots import plot_oa


import warnings
warnings.filterwarnings('ignore')

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
c:\Users\nlab\anaconda3\envs\ephys0\lib\site-packages\deeplabcut\__init__.py:78: UserWarning: 
        As PyTorch is not installed, unsupervised identity learning will not be available.
        Please run `pip install torch`, or ignore this warning.
        
  warnings.warn(


In [2]:
df = pd.read_hdf(r"D:\obstacle_avoidance\recordings\071023\G8CKLN\oadark\testG8CKLN_071023_oadark.h5")

In [4]:
df.condition

AttributeError: 'DataFrame' object has no attribute 'condition'

In [4]:
df.columns

Index(['first_poke', 'second_poke', 'trial_timestamps', 'trial_vidframes',
       'nose_x', 'nose_y', 'nose_likelihood', 'leftear_x', 'leftear_y',
       'leftear_likelihood', 'rightear_x', 'rightear_y', 'rightear_likelihood',
       'spine_x', 'spine_y', 'spine_likelihood', 'midspine_x', 'midspine_y',
       'midspine_likelihood', 'tailbase_x', 'tailbase_y',
       'tailbase_likelihood', 'midtail_x', 'midtail_y', 'midtail_likelihood',
       'tailend_x', 'tailend_y', 'tailend_likelihood', 'arenaTL_x',
       'arenaTL_y', 'arenaTL_likelihood', 'arenaTR_x', 'arenaTR_y',
       'arenaTR_likelihood', 'arenaBL_x', 'arenaBL_y', 'arenaBL_likelihood',
       'arenaBR_x', 'arenaBR_y', 'arenaBR_likelihood', 'leftportT_x',
       'leftportT_y', 'leftportT_likelihood', 'leftportB_x', 'leftportB_y',
       'leftportB_likelihood', 'rightportT_x', 'rightportT_y',
       'rightportT_likelihood', 'rightportB_x', 'rightportB_y',
       'rightportB_likelihood', 'len', 'animal', 'date', 'task', 'nose_x_c

In [5]:
df.head()

,first_poke,second_poke,trial_timestamps,trial_vidframes,nose_x,nose_y,nose_likelihood,leftear_x,leftear_y,leftear_likelihood,...,arenaBR_x_cm,arenaBR_y_cm,leftportT_x_cm,leftportT_y_cm,leftportB_x_cm,leftportB_y_cm,rightportT_x_cm,rightportT_y_cm,rightportB_x_cm,rightportB_y_cm
0,66187.902579,66191.622400,"[66187.903782, 66187.920537, 66187.938048, 661...","[446, 447, 448, 449, 450, 451, 452, 453, 454, ...","[87.59443664550781, 87.5178451538086, 87.15897...","[267.2166748046875, 266.84222412109375, 266.26...","[0.8238534331321716, 0.8239247798919678, 0.778...","[97.57988739013672, 96.82225036621094, 96.0878...","[261.75311279296875, 261.0335693359375, 260.07...","[0.999144434928894, 0.9991301894187927, 0.9988...",...,NaN,NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
1,66191.622400,66194.735360,"[66191.638502, 66191.655756, 66191.672153, 661...","[[670, 671, 672, 673, 674, 675, 676, 677, 678,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.4463430643081665, 0.3927132487297058, 0.245...","[667.9129638671875, 667.6591796875, 668.869079...","[255.01731872558594, 254.76747131347656, 255.0...","[0.9969677329063416, 0.9982151985168457, 0.997...",...,NaN,NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
2,66194.735360,66198.180352,"[66194.740236, 66194.756057, 66194.773107, 661...","[856, 857, 858, 859, 860, 861, 862, 863, 864, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.5557957887649536, 0.381600558757782, 0.3460...","[95.64761352539062, 95.17474365234375, 95.2437...","[263.149169921875, 262.275634765625, 262.62036...","[0.999151349067688, 0.9991017580032349, 0.9989...",...,NaN,NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
3,66198.180352,66200.769190,"[66198.190732, 66198.207718, 66198.225075, 661...","[[1063, 1064, 1065, 1066, 1067, 1068, 1069, 10...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.21104814112186432, 0.14848189055919647, 0.1...","[667.5518188476562, 667.77685546875, 668.03643...","[253.2196502685547, 253.17835998535156, 253.61...","[0.996684730052948, 0.9968668818473816, 0.9967...",...,NaN,NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
4,66200.769190,66203.665280,"[66200.775884, 66200.791705, 66200.808409, 662...","[1218, 1219, 1220, 1221, 1222, 1223, 1224, 122...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.5891939997673035, 0.39691922068595886, 0.37...","[96.66236114501953, 95.6676025390625, 95.29090...","[259.829345703125, 258.8603515625, 258.4705505...","[0.9980910420417786, 0.9977803826332092, 0.996...",...,NaN,NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."


In [4]:
time_thresh = df['len'].quantile(0.9)
df['time'] = df['len']/60
df = df[df['len']<time_thresh]
session = plot_oa('empty',df)
session.single_day_df('obstacle',6)
session.plot_trace_cluster_single_animal()

KeyError: 'start'